In [156]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_curve, roc_auc_score, classification_report

from sklearn.linear_model import LogisticRegression

from sklearn.naive_bayes import GaussianNB

#Classificadores Lineares
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LogisticRegression

#Classificadores KNN
from sklearn.neighbors import KNeighborsClassifier

#Classificadores Naive Nayes
from sklearn.naive_bayes import MultinomialNB

#Classificadores Arvores de Decisão
from sklearn.tree import DecisionTreeClassifier

#SVM
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split

from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_curve, roc_auc_score, classification_report

from sklearn.preprocessing import StandardScaler

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import math

from sklearn import linear_model
from scipy.special import expit

from sklearn.metrics import confusion_matrix
import scipy
from scipy.io import arff

import numpy as np
from sklearn.datasets import fetch_olivetti_faces
from sklearn.svm import SVC
#from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.externals import joblib

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold, cross_val_score, LeaveOneOut

### Agenda
### Pipeline
* Definir um workflow de análise
* Criar um workflow que conecta transformadores e estimadores em uma sequência. 
* Cada etapa é realizada por uma classe, que são encapsuladas em uma classe maior e executadas sob controle de um pipeline
### Salvando e Carregando modelos para arquivos
* joblib
### Busca por hiperparametros
* Grid Search
* Random Search
### Validação Cruzada
* Kfolds
* Kfolds estratificados
* LOOCV
* repetição de divisão entre treino e teste
### Escolha de características (Features)
* kbest

### No código abaixo está sendo feita a comparação entre 3 modelos de classificação

* Preparação dos dados para classificação

In [98]:
X = df.drop(['chd'], axis=1)
y = df['chd']

le = preprocessing.LabelEncoder()
X['famhist'] = le.fit_transform(X['famhist'])

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1)

* Comparação do score dos modelos

In [109]:
sgd = SGDClassifier(random_state=42)
sgd.fit(X_train, y_train)
y_pred = sgd.predict(X_test)
print(sgd.score(X_test, y_test))

tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
print(tree.score(X_test, y_test))

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print(knn.score(X_test, y_test))

0.5319148936170213
0.5319148936170213
0.6382978723404256


* Código complexo e dificil de manter

* Pipelines auxiliam sistematizar a avaliação

### Definindo um workflow com pipeline
* A mesma análise anterior, porem, incluindo cada avaliação em um objeto pipe
* Ao final um pipeline com outros pipes sao executados
* Um pipe possui uma ou mais transformacoes,por exemplo, normalização, padronizaçõ, etc (que possue a funcao transform),  e por último um modelo de predição (que possui a função fit())

In [176]:
# Construindo pipelines
pipe_lr = Pipeline([('scl', StandardScaler()), ('clf', LogisticRegression())])
pipe_knn = Pipeline([('scl', StandardScaler()), ('clf', KNeighborsClassifier())])
pipe_dt = Pipeline([('scl', StandardScaler()), ('clf', DecisionTreeClassifier())])

# Lista de pipelines a serem executados
pipelines = [pipe_lr, pipe_knn, pipe_dt]

# Dicionário para facilitar identificacao
pipe_dict = {0: 'Logistic Regression', 1: 'KNN', 2: 'Decision Tree'}

# aplicando fit
# Generaliza a execucao do fit de cada ultima funcao do pipe
for pipe in pipelines:
    pipe.fit(X_train, y_train)

# Compara acurácia
for idx, val in enumerate(pipelines):
    print('%s pipeline test accuracy: %.3f' % (pipe_dict[idx], val.score(X_test, y_test)))

# para cada modelo treinado obtem val score
best_acc = 0.0
best_clf = 0
best_pipe = ''
for idx, val in enumerate(pipelines):
    # Descobre o melhor val.score e armazen em best_clf
    if val.score(X_test, y_test) > best_acc:
        best_acc = val.score(X_test, y_test)
        best_pipe = val
        best_clf = idx
print('Classifier with best accuracy: %s' % pipe_dict[best_clf])

Logistic Regression pipeline test accuracy: 0.702
KNN pipeline test accuracy: 0.660
Decision Tree pipeline test accuracy: 0.553
Classifier with best accuracy: Logistic Regression


/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


#### Salvando um modelo para um arquivo usando joblib

In [111]:
# Save pipeline to file
joblib.dump(best_pipe, 'best_pipeline.pkl', compress=1)

['best_pipeline.pkl']

#### Carregando um modelo salvo em arquivo

In [112]:
joblib_model= joblib.load('best_pipeline.pkl')

y_pred = joblib_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[26  4]
 [10  7]]


### Busca por hiperparametros
* Hiperparametros sao as diversas parametrizações possíveis de um modelo de predição
* Usando gridsearchCV: busca todas as combinações definidas de hiperparametros, e retorna combinação com o melhor score

In [169]:
# Set the parameters by cross-validation
tuned_parameters = [{'alpha': [1, 10, 100, 1000]},]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()
    
    clf = GridSearchCV(
        RidgeClassifier(), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("classification report:")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'alpha': 10}

Grid scores on development set:

0.679 (+/-0.048) for {'alpha': 1}
0.683 (+/-0.053) for {'alpha': 10}
0.666 (+/-0.065) for {'alpha': 100}
0.666 (+/-0.067) for {'alpha': 1000}

classification report:

              precision    recall  f1-score   support

           0       0.72      0.87      0.79        30
           1       0.64      0.41      0.50        17

    accuracy                           0.70        47
   macro avg       0.68      0.64      0.64        47
weighted avg       0.69      0.70      0.68        47


# Tuning hyper-parameters for recall



/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best parameters set found on development set:

{'alpha': 10}

Grid scores on development set:

0.662 (+/-0.046) for {'alpha': 1}
0.662 (+/-0.049) for {'alpha': 10}
0.644 (+/-0.054) for {'alpha': 100}
0.639 (+/-0.031) for {'alpha': 1000}

classification report:

              precision    recall  f1-score   support

           0       0.72      0.87      0.79        30
           1       0.64      0.41      0.50        17

    accuracy                           0.70        47
   macro avg       0.68      0.64      0.64        47
weighted avg       0.69      0.70      0.68        47




* Usando RandomizedSearchCV: busca um conjunto aleatório de combinações possíveis de hiperparametros, e retorna o modelo com o melhor score

In [170]:
logistic = LogisticRegression(solver='saga', tol=1e-2, max_iter=200, random_state=0)
distributions = dict(penalty=['l2', 'l1'])
clf = RandomizedSearchCV(logistic, distributions, random_state=0)
search = clf.fit(X_test, y_test)
search.best_params_


/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 2 is smaller than n_iter=10. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'penalty': 'l2'}

### Validação Cruzada
#### K-fold
* fold: sub conjunto de dados para teste e treino
* Sub conjuntos gerados movimentando um índice nos dados para definir o início e fim dos dados de treino
* número K define quantidade de movimentações para gerar base de teste

![Folds](https://www.researchgate.net/profile/Mingchao_Li/publication/331209203/figure/fig2/AS:728070977748994@1550597056956/K-fold-cross-validation-method.png)

### Exemplo de Kfold iterando pelos índices

In [121]:
# Amostra de dados
data = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
# Montando Folds
kfold = KFold(4)


for train, test in kfold.split(data):
    print(train , test)
    print(data[train] , data[test])

[2 3 4 5 6 7] [0 1]
[0.3 0.4 0.5 0.6 0.7 0.8] [0.1 0.2]
[0 1 4 5 6 7] [2 3]
[0.1 0.2 0.5 0.6 0.7 0.8] [0.3 0.4]
[0 1 2 3 6 7] [4 5]
[0.1 0.2 0.3 0.4 0.7 0.8] [0.5 0.6]
[0 1 2 3 4 5] [6 7]
[0.1 0.2 0.3 0.4 0.5 0.6] [0.7 0.8]


### Comparando acurácia de classificação com validação cruzada


In [122]:
heart = pd.read_csv('SAheart.data')

le = preprocessing.LabelEncoder()
heart['famhist_label'] = le.fit_transform(heart['famhist'])
heart.drop('famhist', axis=1, inplace=True)

print(heart.head())


X = heart[['sbp','tobacco','ldl','adiposity','typea','obesity','alcohol','age','famhist_label']] 
y = heart[['chd']] 

   sbp  tobacco   ldl  adiposity  typea  obesity  alcohol  age  chd  \
1  160    12.00  5.73      23.11     49    25.30    97.20   52    1   
2  144     0.01  4.41      28.61     55    28.87     2.06   63    1   
3  118     0.08  3.48      32.28     52    29.14     3.81   46    0   
4  170     7.50  6.41      38.03     51    31.99    24.26   58    1   
5  134    13.60  3.50      27.78     60    25.99    57.34   49    1   

   famhist_label  
1              1  
2              0  
3              1  
4              1  
5              1  


In [134]:
sgd = SGDClassifier(random_state=42)
sgd.fit(X_train, y_train)
y_pred = sgd.predict(X_test)
print("Acuracia: %.2f%%" % (sgd.score(X_test, y_test)*100.0)) 

Acuracia: 53.19%


### Teste com Kfolds
* Montando 10 folds para o mesmo classificador testado acima
* cross_val_score aplica os folds e obtem o score do fit de cada sub conjunto
* Ao final mostra a média das acurácias obtidas

In [147]:
kfold = KFold(n_splits=10, random_state=100)
model_kfold = SGDClassifier()
results_kfold = cross_val_score(model_kfold, X_train, y_train, cv=kfold)
print("scores: ", results_kfold) 
print("Acuracia: %.2f%%" % (results_kfold.mean()*100.0)) 

scores:  [0.57142857 0.28571429 0.26190476 0.83333333 0.30952381 0.58536585
 0.63414634 0.7804878  0.58536585 0.6097561 ]
Acuracia: 54.57%


### Folds Stratificados

* Utiliza folds estratificados: cada conjunto contendo aproximadamente a mesma proporção de labels de destino que os dados completos.


In [165]:
skfold = StratifiedKFold(n_splits=6, random_state=100)
model_skfold = SGDClassifier()
results_skfold = cross_val_score(model_skfold, X_train, y_train, cv=skfold)
print("scores: ", results_skfold) 
print("Accuracy: %.2f%%" % (results_skfold.mean()*100.0))

scores:  [0.65714286 0.42857143 0.65217391 0.68115942 0.50724638 0.55882353]
Accuracy: 58.09%


### Leave One Out Cross-Validation (LOOCV)

* Os Fold são definidos com tamanho 1 e K o número de observações

* Essa variação é útil quando os dados de treinamento são de tamanho limitado e o número de parâmetros a serem testados não é alto.

In [166]:
loocv = LeaveOneOut()
model_loocv = SGDClassifier()
results_loocv = cross_val_score(model_loocv, X_train, y_train, cv=loocv)
#print("scores: ", results_loocv) 
print("Accuracy: %.2f%%" % (results_loocv.mean()*100.0))

Accuracy: 61.45%


### Repetição Aleatória de divisão entre treino e teste (Repeated Random Test-Train Splits)

* Híbrido entre divisão tradicional de teste de trem e do método de validação cruzada de k Fold. 

* Nesta técnica, divisões entre treino e teste aleatórias são criadas nos dados da maneira definida pelo conjunto de testes de treinamento 

* Esse processo é repetidos várias vezes, assim como o método de validação cruzada.

In [175]:
kfold2 = ShuffleSplit(n_splits=10, test_size=0.30, random_state=100)
model_shufflecv = SGDClassifier()
results_4 = cross_val_score(model_shufflecv, X_train, y_train, cv=kfold2)
print("scores: ", results_4) 
print("Accuracy: %.2f%% (%.2f%%)" % (results_4.mean()*100.0, results_4.std()*100.0))

scores:  [0.624 0.376 0.328 0.672 0.672 0.432 0.624 0.52  0.736 0.56 ]
Accuracy: 55.44% (13.02%)


In [190]:
#pipe_knn = Pipeline([('scl', StandardScaler()), ('clf', KNeighborsClassifier())])
pipetree = Pipeline([('scl', StandardScaler()), ('clf', DecisionTreeClassifier())])

#pipe = [pipe_knn, pipetre]
pipe = [pipetree]

param_range = [1, 2, 3, 4, 5]

# grid search params
grid_params = [{'clf__criterion': ['gini', 'entropy'],
               'clf__presort': [True, False]}]
#grid_params = [{'clf__criterion': ['gini', 'entropy'],
#    'clf__min_samples_leaf': param_range,
#    'clf__max_depth': param_range,
#    'clf__min_samples_split': param_range[1:],
#    'clf__presort': [True, False]}]

# Construct grid search
gs = GridSearchCV(estimator=pipetree,
    param_grid=grid_params,
    scoring='accuracy')

# Fit using grid search
gs.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs.best_score_)

# Best params
print('\nBest params:\n', gs.best_params_)

Best accuracy: 0.634

Best params:
 {'clf__criterion': 'gini', 'clf__presort': False}


/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


### Escolhendo Features 
* Método kbest
* Utiliza método chi quadrado para escolher melhor conjunto de features de acordo com o seguinte critério:
* Realiza um teste estatístico usando qui-quadrado entre cada features e classe
* O teste do qui-quadrado “elimina” features com maior probabilidade de serem independentes da classe e, portanto, irrelevantes para a classificação.

Fonte: https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html#sklearn.feature_selection.chi2

In [191]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
X, y = load_iris(return_X_y=True)
print(X.shape)
X_new = SelectKBest(chi2, k=2).fit_transform(X, y)
print(X_new.shape)


(150, 4)
(150, 2)


In [194]:
heart = pd.read_csv('SAheart.data')

le = preprocessing.LabelEncoder()
heart['famhist_label'] = le.fit_transform(heart['famhist'])
heart.drop('famhist', axis=1, inplace=True)

print(heart.head())


X = heart[['sbp','tobacco','ldl','adiposity','typea','obesity','alcohol','age','famhist_label']] 
y = heart[['chd']] 

print(X.shape)
X_new = SelectKBest(chi2, k=3).fit_transform(X, y)
print(X_new.shape)


   sbp  tobacco   ldl  adiposity  typea  obesity  alcohol  age  chd  \
1  160    12.00  5.73      23.11     49    25.30    97.20   52    1   
2  144     0.01  4.41      28.61     55    28.87     2.06   63    1   
3  118     0.08  3.48      32.28     52    29.14     3.81   46    0   
4  170     7.50  6.41      38.03     51    31.99    24.26   58    1   
5  134    13.60  3.50      27.78     60    25.99    57.34   49    1   

   famhist_label  
1              1  
2              0  
3              1  
4              1  
5              1  
(462, 9)
(462, 3)
